## Load packages

In [1]:
import rdflib, glob, pandas as pd
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import RDF, RDFS

## Define namespaces

In [2]:
abri = rdflib.Namespace("https://bradleypallen.org/anything-but-routine-ld/4.0/instance/")
abrw = rdflib.Namespace("https://bradleypallen.org/anything-but-routine-ld/4.0/work/")
bf = rdflib.Namespace("http://id.loc.gov/ontologies/bibframe/")
arm = rdflib.Namespace("https://w3id.org/arm/core/ontology/0.1/")

## Parse all edited .ttl files into a graph

In [3]:
def initialize_abr_graph():
    g = rdflib.Graph()
    g.bind("abri", "https://bradleypallen.org/anything-but-routine-ld/4.0/instance/")
    g.bind("abrw", "https://bradleypallen.org/anything-but-routine-ld/4.0/work/")
    g.bind("bf", "http://id.loc.gov/ontologies/bibframe/")
    g.bind("arm", "https://w3id.org/arm/core/ontology/0.1/")
    return g

In [4]:
g = initialize_abr_graph()
for infile in glob.glob("edited-ttl/*/*.ttl"):
    g.parse(infile, format='n3')
n = len(g)
print(f"ABR graph has {n} triples.")

ABR graph has 9666 triples.


## Add Schottlaender identifiers to bf:Works using their URIs

In [8]:
for work in g.subjects(RDF.type, bf.Work):
    uri = work.toPython()
    id = uri[uri.rfind('/')+1:]
    file = f"edited-ttl/work/{id}.ttl"
    h = initialize_abr_graph()
    h.parse(file, format='n3')
    identifier = BNode()
    h.add((work, bf.identifiedBy, identifier))
    h.add((identifier, RDF.type, bf.Identifier))
    h.add((identifier, bf.source, Literal("Schottlaender v4.0")))
    h.add((identifier, RDF.value, Literal(id)))
    h.serialize(file, format='turtle')    